# Continuous training with TFX and Vertex

## Learning Objectives

1. Containerize your TFX code into a pipeline package using Cloud Build.
1. Use the TFX CLI to compile a TFX pipeline.
1. Deploy a TFX pipeline version to run on Vertex Pipelines using the Vertex Python SDK.

### Setup

In [1]:
import os

from google.cloud import aiplatform as vertex_ai

os.environ["PATH"] = f"{os.environ['PATH']}:~/.local/bin"

#### Validate lab package version installation

In [2]:
!python -c "import tensorflow as tf; print(f'TF version: {tf.__version__}')"
!python -c "import tfx; print(f'TFX version: {tfx.__version__}')"
!python -c "import kfp; print(f'KFP version: {kfp.__version__}')"
print(f"aiplatform: {vertex_ai.__version__}")

TF version: 2.8.4
TFX version: 1.8.1
KFP version: 1.8.16
aiplatform: 1.19.0


**Note**: this lab was built and tested with the following package versions:

`TF version: 2.8.4`

`TFX version: 1.8.1` 

`KFP version: 1.8.16`

`aiplatform: 1.19.1`

## Review: example TFX pipeline design pattern for Vertex
The pipeline source code can be found in the `pipeline_vertex` folder.

In [3]:
%cd pipeline_vertex

/home/jupyter/asl-ml-immersion/notebooks/tfx_pipelines/pipeline/labs/pipeline_vertex


In [4]:
!ls -la

total 48
drwxr-xr-x 3 jupyter jupyter 4096 Jul 24 16:02 .
drwxr-xr-x 4 jupyter jupyter 4096 Jul 27 13:27 ..
-rw-r--r-- 1 jupyter jupyter  131 Jul 24 16:02 Dockerfile
-rw-r--r-- 1 jupyter jupyter 1972 Jul 24 16:02 config.py
-rw-r--r-- 1 jupyter jupyter 1090 Jul 24 16:02 features.py
-rw-r--r-- 1 jupyter jupyter 6482 Jul 24 16:02 model.py
-rw-r--r-- 1 jupyter jupyter 5657 Jul 24 16:02 pipeline.py
-rw-r--r-- 1 jupyter jupyter 2182 Jul 24 16:02 preprocessing.py
-rw-r--r-- 1 jupyter jupyter 1322 Jul 24 16:02 runner.py
drwxr-xr-x 2 jupyter jupyter 4096 Jul 24 16:02 schema


The `config.py` module configures the default values for the environment specific settings and the default values for the pipeline runtime parameters. 
The default values can be overwritten at compile time by providing the updated values in a set of environment variables. You will set custom environment variables later on this lab.

The `pipeline.py` module contains the TFX DSL defining the workflow implemented by the pipeline.

The `preprocessing.py` module implements the data preprocessing logic  the `Transform` component.

The `model.py` module implements the TensorFlow model code and training logic for the `Trainer` component.

The `runner.py` module configures and executes `KubeflowV2DagRunner`. At compile time, the `KubeflowDagRunner.run()` method converts the TFX DSL into the pipeline package into a JSON format for execution on Vertex.

The `features.py` module contains feature definitions common across `preprocessing.py` and `model.py`.


## Exercise: build your pipeline with the TFX CLI

You will use TFX CLI to compile and deploy the pipeline. As explained in the previous section, the environment specific settings can be provided through a set of environment variables and embedded into the pipeline package at compile time.

### Configure your environment resource settings

Update  the below constants  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training, Vizier, and Prediction.
- `ARTIFACT_STORE` - An existing GCS bucket. You can use any bucket, but we will use here the bucket with the same name as the project.

In [5]:
# TODO: Set your environment resource settings here for GCP_REGION, ARTIFACT_STORE_URI, ENDPOINT, and CUSTOM_SERVICE_ACCOUNT.
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
ARTIFACT_STORE = f"gs://{PROJECT_ID}"

In [6]:
# Set your resource settings as environment variables. These override the default values in pipeline/config.py.
%env REGION={REGION}
%env ARTIFACT_STORE={ARTIFACT_STORE}
%env PROJECT_ID={PROJECT_ID}

env: REGION=us-central1
env: ARTIFACT_STORE=gs://qwiklabs-asl-04-0aadca72d898
env: PROJECT_ID=qwiklabs-asl-04-0aadca72d898


In [7]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

Creating gs://qwiklabs-asl-04-0aadca72d898/...


### Set the compile time settings to first create a pipeline version without hyperparameter tuning

Default pipeline runtime environment values are configured in the pipeline folder `config.py`. You will set their values directly below:

* `PIPELINE_NAME` - the pipeline's globally unique name.

* `DATA_ROOT_URI` - the URI for the raw lab dataset `gs://{PROJECT_ID}/data/tfxcovertype`.

* `TFX_IMAGE_URI` - the image name of your pipeline container that will be used to execute each of your tfx components

In [8]:
PIPELINE_NAME = "tfxcovertype"
DATA_ROOT_URI = f"gs://{PROJECT_ID}/data/tfxcovertype"
TFX_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{PIPELINE_NAME}"
PIPELINE_JSON = f"{PIPELINE_NAME}.json"

TRAIN_STEPS = 10
EVAL_STEPS = 5

In [9]:
%env PIPELINE_NAME={PIPELINE_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}
%env TFX_IMAGE_URI={TFX_IMAGE_URI}
%env PIPELINE_JSON={PIPELINE_JSON}
%env TRAIN_STEPS={TRAIN_STEPS}
%env EVAL_STEPS={EVAL_STEPS}

env: PIPELINE_NAME=tfxcovertype
env: DATA_ROOT_URI=gs://qwiklabs-asl-04-0aadca72d898/data/tfxcovertype
env: TFX_IMAGE_URI=gcr.io/qwiklabs-asl-04-0aadca72d898/tfxcovertype
env: PIPELINE_JSON=tfxcovertype.json
env: TRAIN_STEPS=10
env: EVAL_STEPS=5


Let us populate the data bucket at `DATA_ROOT_URI`:

In [10]:
!gsutil cp  ../../../data/* $DATA_ROOT_URI/dataset.csv
!gsutil ls $DATA_ROOT_URI/*

Copying file://../../../data/dataset.csv [Content-Type=text/csv]...
/ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      
gs://qwiklabs-asl-04-0aadca72d898/data/tfxcovertype/dataset.csv


Let us build and push the TFX container image described in the `Dockerfile`:

In [11]:
!gcloud builds submit --timeout 15m --tag $TFX_IMAGE_URI .

Creating temporary tarball archive of 14 file(s) totalling 39.6 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-asl-04-0aadca72d898_cloudbuild/source/1690465063.547162-735374f7992c4ea0ae5e59215735b24c.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-04-0aadca72d898/locations/global/builds/71b918d7-dcc2-4bf7-ac1d-bbc3bf63cfde].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/71b918d7-dcc2-4bf7-ac1d-bbc3bf63cfde?project=84583924096 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "71b918d7-dcc2-4bf7-ac1d-bbc3bf63cfde"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-04-0aadca72d898_cloudbuild/source/1690465063.547162-735374f7992c4ea0ae5e59215735b24c.tgz#1690465063805653
Copying gs://qwiklabs-asl-04-0aadca72d898_cloudbuild/source/1690465063.547162-735374f7992c4ea0ae5e59215735b24c.tgz#1690465063805653...
/ [1 files][  8.2 KiB/  8.2 KiB]                            

### Compile your pipeline code

The following command will execute the `KubeflowV2DagRunner` that compiles the pipeline described in `pipeline.py` into a JSON representation consumable by Vertex:

In [14]:
!tfx pipeline compile --engine vertex --pipeline_path runner.py

CLI
Compiling pipeline
running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying features.py -> build/lib
copying model.py -> build/lib
copying runner.py -> build/lib
copying preprocessing.py -> build/lib
copying pipeline.py -> build/lib
copying config.py -> build/lib
/opt/conda/lib/python3.7/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
installing to /var/tmp/tmpubszwaoy
running install
running install_lib
copying build/lib/config.py -> /v

Note: you should see a `{PIPELINE_NAME}.json` file appear in your current pipeline directory.

## Exercise: deploy your pipeline on Vertex using the Vertex SDK

Once you have the `{PIPELINE_NAME}.json` available, you can run the tfx pipeline on Vertex by launching a pipeline job using the `aiplatform` handle:

In [17]:
from google.cloud import aiplatform as vertex_ai

vertex_ai.init(project=PROJECT_ID, location=REGION)

pipeline = vertex_ai.PipelineJob(
    display_name="tfxcovertype4",
    template_path=PIPELINE_JSON,
    enable_caching=False,
)

pipeline.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/84583924096/locations/us-central1/pipelineJobs/tfxcovertype-20230727145124
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/84583924096/locations/us-central1/pipelineJobs/tfxcovertype-20230727145124')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/tfxcovertype-20230727145124?project=84583924096
PipelineJob projects/84583924096/locations/us-central1/pipelineJobs/tfxcovertype-20230727145124 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/84583924096/locations/us-central1/pipelineJobs/tfxcovertype-20230727145124 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/84583924096/locations/us-central1/pipelineJobs/tfxcovertype-20230727145124 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/84583924096/locations/us-central1/pipelineJobs/tfxcovertype-20230727145124

RuntimeError: Job failed with:
code: 13
message: "Internal error happened."


## Next Steps

In this lab, you learned how to build and deploy a TFX pipeline with the TFX CLI and then update, build and deploy a new pipeline with automatic hyperparameter tuning. You practiced triggered continuous pipeline runs using the TFX CLI as well as the Kubeflow Pipelines UI.


In the next lab, you will construct a Cloud Build CI/CD workflow that further automates the building and deployment of the pipeline.

## License

Copyright 2021 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

            http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.